# Selenium CGV 영화 리뷰 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
# url을 바꿔서 못하는경우
# 클릭을 해야지만 바뀌는 경우

!pip install Selenium
!apt-get undate
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

E: Invalid operation undate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (84.0.4147.105-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [2]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def get_movie_reviews(url, page_num=10):

    wd = webdriver.Chrome('chromedriver', options = chrome_options)
    wd.get(url)

    writer_list = []
    review_list = []
    date_list = []

    for page_no in range(1,page_num+1):
        try:
            page_ul = wd.find_element_by_id('paging_point')
            page_a = page_ul.find_element_by_link_text(str(page_no))
            page_a.click()
            #그리고 매번 갱신되게,reflash되게
            time.sleep(1)

            writers = wd.find_elements_by_class_name('writer-name')
            writer_list +=[writer.text for writer in writers]
            
            reviews = wd.find_elements_by_class_name('box-comment')
            review_list += [review.text for review in reviews]
            
            dates = wd.find_elements_by_class_name('day')
            date_list += [date.text for date in dates]

            if page_no % 10 == 0:   # 10페이지 이상일때
                next_button = page_ul.find_elemen_by_class_name('btn-paging next')
                next_button.click()
                time.sleep(1)
        except NoSuchElementException:
            break
    movie_review_df = pd.DataFrame({'writer': writer_list,
                                   'review': review_list,
                                   'date': date_list})
    return movie_review_df

In [3]:
url = 'http://www.cgv.co.kr/movies/detail-view/?midx=83408'
movie_review_df = get_movie_reviews(url, 2)
movie_review_df

,writer,review,date
0,냠냠뇸뇸,가볍게 보기 좋아요. 명절에 가족들과 함께보기 좋은 영화네요,2020.08.30
1,tr**leoto,좋아요 너무좋아요 굿굿,2020.08.30
2,장자봉,볼만했어요 킬링타임용,2020.08.30
3,wo**ung8906,오랫만에 웃음을 주는 영화였어요 아무생각없이 재미있게 볼만해요,2020.08.30
4,콩얄모콩야리모,아무 생각없이 편하게 즐길수 있는 코믹 영화임요. 지루함이 없다는게 큰 장점 ㅎ,2020.08.30
5,rh**nflqkr12,두번째 봤는데도 잼나여!!,2020.08.30
6,김진배반야지,엄정화의 연기와 스토리가 정말 재미있었어요 간만에 유쾌했습니다.,2020.08.30
7,잉느지,오케이마담 킬링타임용,2020.08.30
8,ja**m11,재미있어요 웃을일없는 요즘 간만에 웃었네요 여러사람이 같이봤으면 더 좋았을텐데 큰영...,2020.08.30
9,ss**ang629,속시원한 엄배우님의 액션~,2020.08.30


## CGV 영화 리뷰 긁어오기

* 아이언맨: http://www.cgv.co.kr/movies/detail-view/?midx=38262#1
* 다크나이트: http://www.cgv.co.kr/movies/detail-view/?midx=76417#1
* url을 통해 리뷰 페이지 접근 불가
* 셀레니움으로 페이지 번호를 클릭하여 접근

## CGV 상영작 스크래핑

* http://www.cgv.co.kr/movies/

In [4]:
url = 'http://www.cgv.co.kr/movies/'
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name('sect-movie-chart')
contents = movie_chart.find_elements_by_class_name('box-contents')
for content in contents:
    link = content.find_element_by_tag_name('a').get_attribute('href')
    title = content.find_element_by_class_name('title').text
    percent = content.find_element_by_class_name('percent').text
    info = content.find_element_by_class_name('txt-info').text
    print(title,percent,info,link)
    print(get_movie_reviews(link,2))

테넷 예매율74.8% 2020.08.26 개봉 http://www.cgv.co.kr/movies/detail-view/?midx=83381
        writer                                             review        date
0     효도르얼음파운딩               이해하려고 굳이 애쓰지않아도 자연스럽게 빠져드는 나자신을 발견한다  2020.08.30
1        뷰리풀멑츠                                     세세한 디테일 진한맛 감동  2020.08.30
2      om**527                                   또 봐야 할 것 같은 영화!!  2020.08.30
3   ju**eolove  보다가 어느 순간에 이해가 되기 시작하니 끝난 영화...내용이 아니라 순간순간이 멋...  2020.08.30
4        가연용준맘                                       스릴있고 재미 있었어요  2020.08.30
5    tr**leoto                                       좋아요 너무좋아요 굿굿  2020.08.30
6          디야니  다시 한번 봐야할거같아요 ..피곤한 상태로 갔더니 졸았음 ㅠ 캣 남편 대사 부분 A...  2020.08.30
7         냠냠뇸뇸                        크리스토퍼 놀란 감독은 언제나 진리인 것 같습니다  2020.08.30
8    cl**ierhr                                       보는 내내 이해안됨??  2020.08.30
9         하늘짱?  영화의 이해를 얻기위한 과거의 도도리표 영화를 분석하면서 심호하게 보아야할 포인트 ...  2020.08.30
10  mi**ni0105                      한번에 이해는 안됐지만 믿고보는 감독이라 재밌게봤어